# Automatic Hyperparameter Tuning for DeepScalper
This tutorial shows how automatic hyperparameter tuning is conducted with Optuna library.
## Step 1: Import Packages

In [16]:
import warnings
warnings.filterwarnings("ignore")
import sys
import os

ROOT = os.path.dirname(os.path.abspath("."))
sys.path.append(ROOT)

import torch
import argparse
import os.path as osp
from mmcv import Config
from trademaster.utils import replace_cfg_vals
from trademaster.nets.builder import build_net
from trademaster.environments.builder import build_environment
from trademaster.datasets.builder import build_dataset
from trademaster.agents.builder import build_agent
from trademaster.optimizers.builder import build_optimizer
from trademaster.losses.builder import build_loss
from trademaster.trainers.builder import build_trainer
from trademaster.transition.builder import build_transition
import optuna
from shutil import copyfile
import joblib

from trademaster.utils import set_seed
set_seed(2023)

## Step 2: Load Configs
Load default config from the folder configs/algorithmic_trading/algorithmic_trading_BTC_dqn_dqn_adam_mse.py
Set the flag auto_tuning to True and n_trials as the number of hyperparameter groups wanted to be searched. 

In [17]:
def parse_args():
    parser = argparse.ArgumentParser(description='Download Alpaca Datasets')
    parser.add_argument("--config", default=osp.join(ROOT, "configs", "algorithmic_trading", "algorithmic_trading_BTC_deepscalper_deepscalper_adam_mse.py"),
                        help="download datasets config file path")
    parser.add_argument("--task_name", type=str, default="train")
    parser.add_argument("--auto_tuning", default=False, type=bool)
    parser.add_argument("--n_trials", default=10, type=int)
    args = parser.parse_args([])
    return args

## Step 3: Build RL agent, environment and trainer
Build the environment of algorithmic trading, agent and trainer of DeepScalper following the previous tutorial

In [18]:
def test_dqn_builder(args, sampled_params):
    cfg = Config.fromfile(args.config)
    task_name = args.task_name
    cfg = replace_cfg_vals(cfg)
    
    dataset = build_dataset(cfg)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    train_environment = build_environment(cfg, default_args=dict(dataset=dataset, task="train"))
    valid_environment = build_environment(cfg, default_args=dict(dataset=dataset, task="valid"))
    test_environment = build_environment(cfg, default_args=dict(dataset=dataset, task="test"))

    action_dim = train_environment.action_dim
    state_dim = train_environment.state_dim

    cfg.act.update(dict(action_dim=action_dim, state_dim=state_dim))

    # hyperparameters sampled from sampler 
    if args.auto_tuning == True:
        lr = sampled_params['lr']
        explore_rate = sampled_params['explore_rate']
        dims = sampled_params['dims']
        cfg.optimizer.update(dict(lr=lr))
        cfg.act.update(dict(explore_rate=explore_rate, dims = dims))

    print(cfg)

    act = build_net(cfg.act)
    act_optimizer = build_optimizer(cfg, default_args=dict(params=act.parameters()))

    if cfg.cri:
        cfg.cri.update(dict(action_dim=action_dim, state_dim=state_dim))
        cri = build_net(cfg.cri)
        cri_optimizer = build_optimizer(cfg, default_args=dict(params=cri.parameters()))
    else:
        cri = None
        cri_optimizer = None

    if_remove = cfg.trainer.if_remove
    work_dir = os.path.join(ROOT, cfg.work_dir)
    

    if not os.path.exists(work_dir):
        os.makedirs(work_dir)
    
    cfg.dump(osp.join(ROOT, cfg.work_dir, osp.basename(args.config)))


    criterion = build_loss(cfg)

    transition = build_transition(cfg)

    agent = build_agent(cfg, default_args=dict(action_dim = action_dim,
                                               state_dim = state_dim,
                                               act = act,
                                               cri = cri,
                                               act_optimizer = act_optimizer,
                                               cri_optimizer = cri_optimizer,
                                               criterion = criterion,
                                               transition = transition))

    trainer = build_trainer(cfg, default_args=dict(train_environment = train_environment,
                                               valid_environment = valid_environment,
                                               test_environment = test_environment,
                                               agent = agent,
                                               device = device))
    return trainer

## Step 4: Sample hyperparameters using Optuna
Optuna library provides functions to sample different types of hyperparameters. In this tutorial, learning rate, explore rate, number of hidden nodes of DQN are tuned. 

In [19]:
def sample_params(trial: optuna.Trial):
    lr = trial.suggest_float("lr", 1e-4, 1e-3, log=True)
    explore_rate = trial.suggest_float("explore_rate", 0.15, 0.3, step=0.05)
    dims = trial.suggest_categorical("hidden_nodes", [(64,32), (128,64)])
    trial_number = trial.number
    sampled_params = dict(lr=lr, explore_rate=explore_rate, dims=dims, trial_number=trial_number)
    return sampled_params

## Step 5: Define objective function of the optimization procedure
Optuna tries to find the best hyperparameter pair by optimizing a specific objective function. 
In this tutorial, the original training procedure is repeated, and the sum of episode reward on valid dataset is used as the objective function. 

In [20]:
def objective(trial: optuna.Trial) -> float:
    args = parse_args()
    sampled_params = sample_params(trial)
    trainer = test_dqn_builder(args, sampled_params)
    valid_score = trainer.train_and_valid_trial(sampled_params['trial_number'])
    return valid_score

## Step 6: Run the whole tuning procedure
Due to the fact that the original training pipeline will be run for n_trials of times, it might take time to be finished. Please use smaller n_trials to reduce time for tuning.

In [21]:
args = parse_args()
cfg = Config.fromfile(args.config)

if args.task_name.startswith("train"):
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=args.n_trials)
    print("Best trial:")
    trial = study.best_trial
    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    print("  Best trial number: ", trial.number)
    best_model_path = os.path.join(ROOT, cfg.work_dir, "checkpoints", "trial-{:05d}.pth".format(trial.number))
    copyfile(best_model_path, os.path.join(ROOT, cfg.work_dir, "checkpoints", "best.pth"))
    joblib.dump(study, os.path.join(ROOT, cfg.work_dir, "auto_tuning.pkl"))
    trainer = test_dqn_builder(args, sampled_params = sample_params(study.best_trial))
    trainer.test()
    print("train end")
elif args.task_name.startswith("test"):
    study = joblib.load(os.path.join(ROOT, cfg.work_dir, "auto_tuning.pkl"))
    trainer = test_dqn_builder(args, sampled_params = sample_params(study.best_trial))
    trainer.test()
    print("test end")

[I 2023-02-28 02:55:16,121] A new study created in memory with name: no-name-54189c06-f279-4b3f-9f16-e8fb422e9dd5


Config (path: E:\workspace\RA\TradeMaster\configs\algorithmic_trading\algorithmic_trading_BTC_deepscalper_deepscalper_adam_mse.py): {'data': {'type': 'AlgorithmicTradingDataset', 'data_path': 'data/algorithmic_trading/BTC', 'train_path': 'data/algorithmic_trading/BTC/train.csv', 'valid_path': 'data/algorithmic_trading/BTC/valid.csv', 'test_path': 'data/algorithmic_trading/BTC/test.csv', 'test_dynamic_path': 'data/algorithmic_trading/BTC/test_labeled_3_24_-0.15_0.15.csv', 'tech_indicator_list': ['high', 'low', 'open', 'close', 'adjcp', 'zopen', 'zhigh', 'zlow', 'zadjcp', 'zclose', 'zd_5', 'zd_10', 'zd_15', 'zd_20', 'zd_25', 'zd_30'], 'backward_num_day': 5, 'forward_num_day': 5, 'test_dynamic': '-1'}, 'environment': {'type': 'AlgorithmicTradingEnvironment'}, 'agent': {'type': 'AlgorithmicTradingDQN', 'max_step': 12345, 'reward_scale': 1, 'repeat_times': 1, 'gamma': 0.9, 'batch_size': 64, 'clip_grad_norm': 3.0, 'soft_update_tau': 0, 'state_value_tau': 0.005}, 'trainer': {'type': 'Algorith

[I 2023-02-28 02:58:57,868] Trial 0 finished with value: 58568.12801411601 and parameters: {'lr': 0.0009589693908250198, 'explore_rate': 0.2, 'hidden_nodes': (64, 32)}. Best is trial 0 with value: 58568.12801411601.


+--------------+-------------+------------+--------------+
| Total Return | Sharp Ratio | Volatility | Max Drawdown |
+--------------+-------------+------------+--------------+
| -169.942505% |  -2.789733  | 56.884068% | 190.033557%  |
+--------------+-------------+------------+--------------+
Config (path: E:\workspace\RA\TradeMaster\configs\algorithmic_trading\algorithmic_trading_BTC_deepscalper_deepscalper_adam_mse.py): {'data': {'type': 'AlgorithmicTradingDataset', 'data_path': 'data/algorithmic_trading/BTC', 'train_path': 'data/algorithmic_trading/BTC/train.csv', 'valid_path': 'data/algorithmic_trading/BTC/valid.csv', 'test_path': 'data/algorithmic_trading/BTC/test.csv', 'test_dynamic_path': 'data/algorithmic_trading/BTC/test_labeled_3_24_-0.15_0.15.csv', 'tech_indicator_list': ['high', 'low', 'open', 'close', 'adjcp', 'zopen', 'zhigh', 'zlow', 'zadjcp', 'zclose', 'zd_5', 'zd_10', 'zd_15', 'zd_20', 'zd_25', 'zd_30'], 'backward_num_day': 5, 'forward_num_day': 5, 'test_dynamic': '-1

[W 2023-02-28 02:59:14,042] Trial 1 failed with parameters: {'lr': 0.0003576932787709103, 'explore_rate': 0.25, 'hidden_nodes': (64, 32)} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "D:\ProgramData\Anaconda3\envs\python3.9\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\zwt\AppData\Local\Temp\ipykernel_13020\546600894.py", line 5, in objective
    valid_score = trainer.train_and_valid_trial(sampled_params['trial_number'])
  File "E:\workspace\RA\TradeMaster\trademaster\trainers\algorithmic_trading\trainer.py", line 214, in train_and_valid_trial
    logging_tuple = self.agent.update_net(buffer)
  File "E:\workspace\RA\TradeMaster\trademaster\agents\algorithmic_trading\dqn.py", line 152, in update_net
    obj_critic, q_value = self.get_obj_critic(buffer, self.batch_size)
  File "E:\workspace\RA\TradeMaster\trademaster\agents\algorithmic_trading\dqn.py", line 121, in

KeyboardInterrupt: 